In [1]:
### start_imports ###
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
### end_imports ###
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### start_miscellaneous ###
from datetime import date,datetime,timedelta
hoje =  pd.to_datetime(date.today()) 

final_do_mes = pd.to_datetime(datetime(hoje.year,hoje.month+1,1) - timedelta(seconds=1))

biu_download_cols  = ['vmonitoramentoleilao', 'vmonitoramentoug', 'vmonitoramentousina', 'vrapeelacesso', 'vrapeelcontratorecurso', 'vrapeelcronograma', 'vrapeelempreendimento', 'vrapeellicenciamento', 'vrapeeloperacaoug']
biu_path = os.path.join(get_standard_folder_path("Documents"),"Previsor/BIU")

def ultimo_rapeel(df):
    df_ultimo_envio = df.loc[df.groupby("IdeUsinaOutorga")['DthEnvio'].idxmax()][['IdeUsinaOutorga','DthEnvio']]
    return pd.merge(df_ultimo_envio,df,on=["IdeUsinaOutorga","DthEnvio"],how="left")


list_carac_usina = ['Usina_Sem_Rapeel','CondicaoAmbiental','Homologar_Marcos','Revisar_IO','Usina_Selecionada','Paralisada','Sem_Previsao','criterionovopmo','Prev_OC_passado','Em_teste','Sem_Monitoramento','Manual','UG_Sem_Rapeel']
list_carac_ug = ['FASE','flagOPTeste30dias','UG_Sem_Rapeel','criterionovopmo']
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']
list_ambiental = ['DatValidadeLI','DatValidadeLP','DatValidadeLO','ValidadeAmbiental','CondicaoAmbiental']
list_contratos = ['NomSitContratoCCD', 'NomSitContratoCUST', 'NomSituacaoContratoCUSD', 'NomSitContratoCCT']
### end_miscellaneous ###

Current working directory was changed to: C:\Users\Vinicius\Documents\GitHub\Previsor\App


In [2]:
#### start_function ### function_name = generate_BIU
### start_function_vars ###
### start_funcion_defined_vars ###
### end_funcion_defined_vars ###
biu_download_path = os.path.join(biu_path,"Download")
previsao_file = os.path.join(biu_path,"Estagio_I/Previsao_OC.gzip")
### end_function_vars ###

In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [4]:
cols = ["IdeUsinaOutorga",'NomUsina','CodCegFormatado','DatMonitoramento',"DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado",'DscJustificativaPrevisao']
# Carrega informações do monitoramento
df_usina = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentousina.gzip"))[cols]

#Transforma coluna em booleano
df_usina.IdcUsinaMonitorada = df_usina.IdcUsinaMonitorada.map({"Sim":"True","Não":False}).astype(bool)

# Remove usinas que não são monitoradas
df_usina = df_usina[df_usina.IdcUsinaMonitorada].reset_index(drop=True)

In [5]:
# Excluir ? Não está sendo usado
monitoramentoleilao  = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
# Mantém apenas primeira data de suprimento 
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [6]:
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(biu_download_path,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Faz merge left entre as informações do monitoramento e do rapeel
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [7]:
# Cria coluna para identificar usinas que não enviaram rapeel
df_usina["Usina_Sem_Rapeel"] = True
df_usina.loc[df_usina.IdeUsinaOutorga.isin(rapeel_cronograma.IdeUsinaOutorga),'Usina_Sem_Rapeel'] = False

In [8]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(biu_download_path,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [9]:
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(biu_download_path,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [10]:
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(biu_download_path,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [11]:
df_usina["classe"] = df_usina.CodCegFormatado.str.slice(4,6)
df_usina["CC_real"] = pd.to_datetime(pd.NA)
df_usina.loc[df_usina.classe == "PH","CC_real"] = df_usina.DatRealizacaoIV
df_usina.loc[df_usina.classe == "CV","CC_real"] = df_usina.DatRealizacaoV
df_usina["ME_real"] = df_usina.DatRealizacaoX
df_usina.loc[df_usina.classe == "CV","ME_real"] = df_usina.DatRealizacaoVI

In [12]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
cols = ['IdeUsinaOutorga'] + list(dict_validades.keys()) + ['ValidadeAmbiental']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental,IdeUsinaOutorga (case),DatValidadeLO (case),DatValidadeLI (case),DatValidadeLP (case),ValidadeAmbiental (case),Count
2,659,2026-02-22,2028-06-28,2000-03-29,2026-02-22,True,True,True,True,True,771
0,324,NaT,2026-03-09,2002-12-11,2026-03-09,True,False,True,True,True,741
25,28616,NaT,NaT,2013-05-21,2013-05-21,True,False,False,True,True,642
1,601,NaT,NaT,NaT,NaT,True,False,False,False,False,313
2302,54659,NaT,2026-10-22,NaT,2026-10-22,True,False,True,False,True,7
7,1997,2024-01-22,NaT,2011-11-17,2024-01-22,True,True,False,True,True,3


In [13]:
df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'

cols = ['IdeUsinaOutorga'] +  list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']

display(show_cases(df_usina,cols))

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"

cols = ['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']

display(show_cases(df_usina,cols))



df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"


cols = ['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoUso']
display(show_cases(df_usina,cols))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

cols = ['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental,CondicaoAmbiental,IdeUsinaOutorga (case),DatValidadeLO (case),DatValidadeLI (case),DatValidadeLP (case),ValidadeAmbiental (case),CondicaoAmbiental (case),Count
2,659,2026-02-22,2028-06-28,2000-03-29,2026-02-22,LO,True,True,True,True,True,LO,771
0,324,NaT,2026-03-09,2002-12-11,2026-03-09,LI,True,False,True,True,True,LI,741
25,28616,NaT,NaT,2013-05-21,2013-05-21,LP,True,False,False,True,True,LP,642
3,1195,NaT,NaT,NaT,NaT,Não Informado,True,False,False,False,False,Não Informado,244
1,601,NaT,NaT,NaT,NaT,Sem LP,True,False,False,False,False,Sem LP,69
2302,54659,NaT,2026-10-22,NaT,2026-10-22,LI,True,False,True,False,True,LI,7
7,1997,2024-01-22,NaT,2011-11-17,2024-01-22,LO,True,True,False,True,True,LO,3


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao,IdeUsinaOutorga (case),NomSitContratoCCD (case),NomSitContratoCCT (case),DthEnvio (case),CondicaoConexao (case),Count
12,27835,Não se Aplica,Não Assinado,2023-01-04 20:32:15.297,Sem Conexão,True,Não se Aplica,Não Assinado,True,Sem Conexão,1084
0,324,Não se Aplica,Válido,2023-01-02 08:34:07.347,OK,True,Não se Aplica,Válido,True,OK,764
3,1195,NaN,NaN,NaT,Não informado,True,NaN,NaN,False,Não informado,244
7,1997,Não se Aplica,Não se Aplica,2023-01-02 14:04:26.367,Verificar,True,Não se Aplica,Não se Aplica,True,Verificar,151
6,1928,Válido,Não se Aplica,2023-01-05 19:34:10.547,OK,True,Válido,Não se Aplica,True,OK,98
1,601,Não Assinado,Não se Aplica,2023-01-03 19:22:43.140,Sem Conexão,True,Não Assinado,Não se Aplica,True,Sem Conexão,81
25,28616,Não Assinado,Não Assinado,2023-01-02 16:42:33.840,Sem Conexão,True,Não Assinado,Não Assinado,True,Sem Conexão,51
2,659,Válido,Válido,2023-01-06 09:37:52.703,OK,True,Válido,Válido,True,OK,2
619,38369,Não se Aplica,Vencido,2023-01-02 17:59:56.687,Sem Conexão,True,Não se Aplica,Vencido,True,Sem Conexão,1
41,29505,Vencido,Não se Aplica,2023-01-05 12:39:59.160,Sem Conexão,True,Vencido,Não se Aplica,True,Sem Conexão,1


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoUso,IdeUsinaOutorga (case),NomSituacaoContratoCUSD (case),NomSitContratoCUST (case),CondicaoUso (case),Count
50,30105,Não se Aplica,Válido,OK,True,Não se Aplica,Válido,OK,936
46,29716,Não se Aplica,Não Assinado,Sem Uso,True,Não se Aplica,Não Assinado,Sem Uso,844
3,1195,NaN,NaN,Não informado,True,NaN,NaN,Não informado,244
0,324,Válido,Não se Aplica,OK,True,Válido,Não se Aplica,OK,169
7,1997,Não se Aplica,Não se Aplica,Verificar,True,Não se Aplica,Não se Aplica,Verificar,114
1,601,Não Assinado,Não se Aplica,Sem Uso,True,Não Assinado,Não se Aplica,Sem Uso,99
25,28616,Não Assinado,Não Assinado,Sem Uso,True,Não Assinado,Não Assinado,Sem Uso,52
2,659,Válido,Válido,OK,True,Válido,Válido,OK,11
264,34829,Não se Aplica,Vencido,Sem Uso,True,Não se Aplica,Vencido,Sem Uso,3
18,28000,Vencido,Vencido,Sem Uso,True,Vencido,Vencido,Sem Uso,2


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA,IdeUsinaOutorga (case),DscComercializacaoEnergia (case),DatConclusaoACL (case),PPA (case),Count
0,324,Fora do ACR,NaT,Nenhum,True,Fora do ACR,False,Nenhum,1964
34,29116,Fora do ACR,2022-03-04,ACL,True,Fora do ACR,True,ACL,252
2,659,ACR,NaT,ACR,True,ACR,False,ACR,220
44,29576,ACR,2013-04-03,Ambos,True,ACR,True,Ambos,41


In [14]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

In [15]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


cols = ['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo']
display(show_cases(df_usina,cols,max_cases=10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,IdeUsinaOutorga (case),IdcSituacaoObra (case),PPA (case),CondicaoAmbiental (case),CondicaoUso (case),criterio_novo (case),Count
29,28791,Não Iniciada,Nenhum,LP,Sem Uso,5,True,Não Iniciada,Nenhum,LP,Sem Uso,5,342
93,32076,Não Iniciada,Nenhum,LO,Sem Uso,3,True,Não Iniciada,Nenhum,LO,Sem Uso,3,316
156,32996,Não Iniciada,Nenhum,LO,OK,2,True,Não Iniciada,Nenhum,LO,OK,2,272
99,32145,Não Iniciada,Nenhum,Não Informado,Não informado,7,True,Não Iniciada,Nenhum,Não Informado,Não informado,7,240
43,29526,Não Iniciada,Nenhum,LI,Sem Uso,3,True,Não Iniciada,Nenhum,LI,Sem Uso,3,184
49,30101,Não Iniciada,Nenhum,LP,OK,4,True,Não Iniciada,Nenhum,LP,OK,4,172
10,27165,Não Iniciada,Nenhum,LI,OK,2,True,Não Iniciada,Nenhum,LI,OK,2,169
67,30666,Em andamento,Nenhum,LI,OK,0,True,Em andamento,Nenhum,LI,OK,0,75
34,29116,Em andamento,ACL,LO,OK,0,True,Em andamento,ACL,LO,OK,0,74
109,32305,Em andamento,ACL,LI,OK,0,True,Em andamento,ACL,LI,OK,0,62


In [16]:
dict_marcos_homologar = {
    'DatInicioObraRealizado' : 'IO_real',
    'DatConcretagemRealizado' : 'CC_real',
    'DatMontagemRealizado' : 'ME_real',
    'DatEnchimentoRealizado' : 'Ench_Real',
    'DatSisTransmissaoRealizado' : 'IOTrans_Real',
    'DatCanteiroObraRealizado' : 'canteiroReal',
    'DatComissionamentoUGRealizado' : 'Comiss_Real',
    'DatConclusaoSisTransRealizado' : 'Conc_Trans_REAL',
    'DatDesvioRioRealizado' : 'DesvRio_real',
    'DatConclusaoTorresRealizado' : 'ME_Real_conc_eol'}
    
df_usina['Homologar_Marcos'] = False
df_usina['Dsc_Marcos_a_Homologar'] = ''

# Homologação de marcos

for marco_monitoramento, marco_rapeel in dict_marcos_homologar.items():
    mask = (df_usina[marco_monitoramento].isna()) & (df_usina[marco_rapeel].notna())
    df_usina['Homologar_Marcos'] |= mask
    df_usina.loc[mask,'Dsc_Marcos_a_Homologar'] += f'{marco_monitoramento}, '

df_usina.Dsc_Marcos_a_Homologar = df_usina.Dsc_Marcos_a_Homologar.str.slice(0,-2)

cols = ["IdeUsinaOutorga",'NomUsina'] + ['Homologar_Marcos','Dsc_Marcos_a_Homologar']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,NomUsina,Homologar_Marcos,Dsc_Marcos_a_Homologar,IdeUsinaOutorga (case),NomUsina (case),Homologar_Marcos (case),Dsc_Marcos_a_Homologar (case),Count
0,324,Bugres,False,,True,True,False,True,2367
8,2586,Salto Cafesoca,True,DatSisTransmissaoRealizado,True,True,True,True,110


In [17]:
df_usina['Revisar_IO'] =  np.select(
 [(df_usina.DatPrevisaoIniciobra < (hoje + pd.Timedelta(15,"D"))) |
 (df_usina.prev_IO > df_usina.DatPrevisaoIniciobra)],
 [True],
 default = False)
cols = ['IdeUsinaOutorga', 'NomUsina','DatPrevisaoIniciobra']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,NomUsina,DatPrevisaoIniciobra,IdeUsinaOutorga (case),NomUsina (case),DatPrevisaoIniciobra (case),Count
10,27165,Emas Nova,2023-03-01,True,True,True,1969
0,324,Bugres,NaT,True,True,False,508


In [18]:
usinas_selecionadas = read_file("./usinas_selecionadas.txt").split(",")
usinas_selecionadas = [int(num) for num in usinas_selecionadas]
df_usina["Usina_Selecionada"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(usinas_selecionadas),'Usina_Selecionada'] = True

df_usina['Paralisada'] = False
df_usina.loc[df_usina.IdcObraParalisada=="Sim",'Paralisada'] = True

df_usina['Sem_Previsao'] = False
df_usina.loc[df_usina.IdcSemPrevisao=="Sim",'Sem_Previsao'] = True

cols = ['IdeUsinaOutorga', 'NomUsina','Usina_Selecionada','Paralisada','Sem_Previsao']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,NomUsina,Usina_Selecionada,Paralisada,Sem_Previsao,IdeUsinaOutorga (case),NomUsina (case),Usina_Selecionada (case),Paralisada (case),Sem_Previsao (case),Count
6,1928,Pacífico Mascarenhas,False,False,False,True,True,False,False,False,2297
0,324,Bugres,False,False,True,True,True,False,False,True,86
11,27243,Santa Cruz,True,False,False,True,True,True,False,False,47
15,27969,Itaocara I,True,False,True,True,True,True,False,True,27
2,659,Capivari,False,True,True,True,True,False,True,True,13
53,30150,Almirante Álvaro Alberto - Unidade III (Antiga...,True,True,True,True,True,True,True,True,6
597,38221,Galvani Serra do Salitre,False,True,False,True,True,False,True,False,1


# Informações de UGs

## Carrega banco de dados

In [19]:
cols = ["IdeUsinaOutorga",'NumUgUsina',"IdcMonitorada",'DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})
# Não remover ugs não monitoradas nesse passo, pois serão usadas para encontrar ugs difentes entre 
# monitoramento e rapeel. São retiradas durante o merge
monitoramentoug.IdcMonitorada = monitoramentoug.IdcMonitorada.map({"Sim":True,"Não":False}).astype(bool)

display(show_cases(monitoramentoug,monitoramentoug.columns))

,IdeUsinaOutorga,NumOperacaoUg,IdcMonitorada,DatLiberOpComerRealizado,IdeUsinaOutorga (case),NumOperacaoUg (case),IdcMonitorada (case),DatLiberOpComerRealizado (case),Count
2,1358,1,True,NaT,True,True,True,False,74652
0,742,1,False,1962-12-14,True,True,False,True,15130
9,27422,3,False,NaT,True,True,False,False,854


In [20]:
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(biu_download_path,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ultimo_rapeel(ug_rapeel)
display(show_cases(ug_rapeel,ug_rapeel.columns))

,IdeUsinaOutorga,DthEnvio,NumOperacaoUg,DatPrevistaComercial,IdeUsinaOutorga (case),DthEnvio (case),NumOperacaoUg (case),DatPrevistaComercial (case),Count
0,273,2022-10-09 21:04:17.743,4,2023-11-02,True,True,True,True,81877
6,659,2023-01-06 09:37:52.703,2,NaT,True,True,True,False,2562


In [21]:
cols_used_robot = ['DatInicioObraOutorgado','DatPrevisaoIniciobra','prev_IO']
list_cols_used =   ['IdeUsinaOutorga','IdcObraParalisada','IdcSemPrevisao','criterio_novo','Usina_Sem_Rapeel',] + cols_used_robot
# UGs não monitoradas são retiradas aqui
df_ug = pd.merge(df_usina[list_cols_used],monitoramentoug[monitoramentoug.IdcMonitorada],on="IdeUsinaOutorga",how='left')
df_ug.IdcMonitorada = df_ug.IdcMonitorada.astype(bool)
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left') # poderia ser inner?
# Remove usinas que já entraram em operação comercial ou usinas sem UGs monitoradas
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna() &  df_ug.NumOperacaoUg.notna()].reset_index(drop=True)

In [22]:
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)[['IdeUsinaOutorga','NumOperacaoUg','FASE','Ind_crono_norm','flagOPTeste30dias','Previsao_OC','Dat_OC_obrigacao','MdaPotenciaUnitaria']] 

df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")
display(show_cases(calculo_previsao,calculo_previsao.columns))

,IdeUsinaOutorga,NumOperacaoUg,FASE,Ind_crono_norm,flagOPTeste30dias,Previsao_OC,Dat_OC_obrigacao,MdaPotenciaUnitaria,IdeUsinaOutorga (case),NumOperacaoUg (case),FASE (case),Ind_crono_norm (case),flagOPTeste30dias (case),Previsao_OC (case),Dat_OC_obrigacao (case),MdaPotenciaUnitaria (case),Count
0,1358,1,OUT,4.650318e-03,0,2025-07-15 13:24:30.102,2016-02-01,3500.0,True,True,OUT,True,0,True,True,True,62618
15,30482,3,IO,8.458899e-01,0,2024-02-14 10:47:40.782,2025-01-01,16000.0,True,True,IO,True,0,True,True,True,5719
8,28818,2,ME,8.970399e-01,0,2023-05-25 20:53:40.592,2025-01-01,11151.0,True,True,ME,True,0,True,True,True,4664
83,31685,8,OT,5.922102e-07,0,2023-02-03 07:58:45.697,2019-03-01,3437.0,True,True,OT,True,0,True,True,True,827
10,29433,2,CC,7.106852e-01,0,2024-01-27 08:37:14.522,2023-07-31,4500.0,True,True,CC,True,0,True,True,True,533
22,27243,1,OT,4.121207e-04,1,2023-04-11 09:25:05.356,2014-12-01,150000.0,True,True,OT,True,1,True,True,True,291


In [23]:
monitoramentoug['Existe_Monitoramento'] = True     # Adiciona colunas para checagem
ug_rapeel['Existe_Rapeel'] = True                  # dos resultados

# Faz merge tipo outer (todas as UGs) com os dados do  
# monitoramento e rapeel

comp_ug = pd.merge(monitoramentoug[list_id_ug + ['Existe_Monitoramento',"IdcMonitorada"]], 
        ug_rapeel[list_id_ug + ['Existe_Rapeel']], how="outer")                            

# Inner join com usinas monitoradas
comp_ug = pd.merge(df_usina[df_usina.IdcUsinaMonitorada][['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga")

# Inner join com usinas que já enviaram rapeel
comp_ug = pd.merge(rapeel_cronograma[['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga") 

# Das UGs que estão no monitoramento, remove-se UGs não monitoradas
comp_ug = comp_ug[comp_ug.IdcMonitorada != False]

# Preenche com o valor falso as usinas que não foram encontradas
comp_ug[['Existe_Monitoramento','Existe_Rapeel']] = comp_ug[['Existe_Monitoramento','Existe_Rapeel']].fillna(False) 

# Coluna com diferenças
comp_ug["Diff"] = ~(comp_ug.Existe_Monitoramento & comp_ug.Existe_Rapeel) 


df_UG_Sem_Rapeel = comp_ug[comp_ug.Existe_Monitoramento & comp_ug.Diff][['IdeUsinaOutorga']].drop_duplicates()
df_UG_Sem_Rapeel['UG_Sem_Rapeel'] = True


# Todas as UGs de uma usina que pelo menos uma UG não possua rapeel terão UG_Sem_Rapeel = True
df_ug = pd.merge(df_ug,df_UG_Sem_Rapeel,on='IdeUsinaOutorga',how='left')
df_ug.loc[df_ug.UG_Sem_Rapeel.isna(),'UG_Sem_Rapeel'] = False


df_ug = df_ug.merge(ug_rapeel.loc[ug_rapeel[ug_rapeel.DatPrevistaComercial.notna()].groupby("IdeUsinaOutorga").DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial']].rename(columns={'DatPrevistaComercial':'PrevisaoOC_rapeel_max'}),
            how="left",on="IdeUsinaOutorga")




# Para esses casos, em vez de DatPrevistaComercial, será usado PrevisaoOC_rapeel_max
# Ver arquivo 
df_ug.loc[df_ug.UG_Sem_Rapeel | df_ug.DatPrevistaComercial.isna(),'DatPrevistaComercial'] = df_ug.PrevisaoOC_rapeel_max


##  Cria colunas com definições e regras

In [24]:
################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)

################################ regranovapmo ################################

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)

df_ug["Handicap"] = hoje + pd.Timedelta(5*365,'D')
df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Handicap']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Handicap"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Handicap']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA

del df_ug['Handicap']

In [25]:
serie_usinas_previsao_OC_passado = df_ug[(df_ug.Previsao_OC < final_do_mes) | (df_ug.Previsao_OC.isna())].IdeUsinaOutorga.drop_duplicates()
serie_usinas_previsao_em_teste = df_ug[df_ug.criterionovopmo.isin([0.2,0.3])].IdeUsinaOutorga.drop_duplicates()

serie_flagOPTeste30dias = df_ug[df_ug.flagOPTeste30dias == 1].IdeUsinaOutorga.drop_duplicates()
df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

serie_UG_Sem_Rapeel = df_ug[df_ug.UG_Sem_Rapeel].IdeUsinaOutorga.drop_duplicates()

In [26]:
df_usina["UG_Sem_Rapeel"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_UG_Sem_Rapeel),'UG_Sem_Rapeel'] = True

df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

df_usina["Prev_OC_passado"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_OC_passado),'Prev_OC_passado'] = True

df_usina["Em_teste"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_em_teste),'Em_teste'] = True

df_usina["Sem_Monitoramento"] = False
df_usina.loc[df_usina.DatMonitoramento < (hoje - pd.Timedelta(4*30, unit="D")),'Sem_Monitoramento'] = True

In [27]:
df_usina["Manual"] = False
df_usina.loc[(df_usina.Paralisada | df_usina.Sem_Previsao | 
    df_usina.Usina_Selecionada| df_usina.Prev_OC_passado | df_usina.Em_teste | 
    (df_usina.DscJustificativaPrevisao == 'Situação das obras de conexão e linha de transmissão associada.')),
    'Manual'] = True

In [28]:
# Casos de seleção

df_usina['Caso_I'] = df_usina['Caso_II_a'] = df_usina['Caso_II_b'] = df_usina['Caso_III'] = False
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']

df_usina['Caso_I'] = (~df_usina.Manual) & (~df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos) 

df_usina['Caso_II_a'] = (~df_usina.Manual) & (df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos)

df_usina['Caso_II_b'] = (~df_usina.Manual)  & ((df_usina.Homologar_Marcos) | (df_usina.Homologar_Marcos & df_usina.Revisar_IO))

df_usina['Caso_III'] = (df_usina.Manual & (df_usina.Sem_Monitoramento | df_usina.Homologar_Marcos | df_usina.Revisar_IO)) | df_usina.Em_teste


df_usina['Selecionado_BIU'] = df_usina[list_casos].any(axis=1)

In [29]:
list_criterios = ['criterio_novo',"criterionovopmo"]

df_usina_criterio = df_ug.loc[df_ug.groupby("IdeUsinaOutorga").criterionovopmo.idxmax()][["IdeUsinaOutorga","FASE",'flagOPTeste30dias'] + list_criterios]

################################ dscjustificativaregranova ################################
list_condicoes = [
    (df_usina_criterio['criterionovopmo'] == 9),
    (df_usina_criterio['criterionovopmo'] == 8),
    (df_usina_criterio['criterionovopmo'] == 0.2),
    (df_usina_criterio['criterionovopmo'] == 0.1) | (df_usina_criterio['criterionovopmo'] == 0.3),
    (df_usina_criterio['criterionovopmo'] == 1),
    df_usina_criterio['criterionovopmo']==2,
    df_usina_criterio['criterionovopmo']==3,
    (df_usina_criterio.criterionovopmo== 4),
    (df_usina_criterio.criterionovopmo== 5),
    (df_usina_criterio.criterionovopmo== 6),
    (df_usina_criterio.criterionovopmo== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_usina_criterio['dscjustificativaregranova'] = np.select(list_condicoes,list_values)

################################ dsccriterionovo ################################
list_condicoes = [
    ((df_usina_criterio['criterio_novo'] == 9)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] != "OT")),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT") & (df_usina_criterio['flagOPTeste30dias'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT")),
    ((df_usina_criterio['criterio_novo'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 2)),
    ((df_usina_criterio['criterio_novo'] == 3)),
    ((df_usina_criterio['criterio_novo'] == 4)),
    ((df_usina_criterio['criterio_novo'] == 5)),
    ((df_usina_criterio['criterio_novo'] == 7)),
    ((df_usina_criterio['criterio_novo'] == 6)),
    ((df_usina_criterio['criterio_novo'] == 8))
]

list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_usina_criterio['dsccriterionovo'] = np.select(list_condicoes,list_values)
list_justificativas = ['dsccriterionovo','dscjustificativaregranova']

In [30]:
df_usina = pd.merge(df_usina,
df_usina_criterio[['IdeUsinaOutorga','criterionovopmo'] + list_justificativas],
on = "IdeUsinaOutorga", how='left')


list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III','Selecionado_BIU']
df_ug = pd.merge(df_usina[['IdeUsinaOutorga','criterionovopmo','dscjustificativaregranova','dsccriterionovo','DatMonitoramento','DscJustificativaPrevisao'] + list_casos],df_ug, on = "IdeUsinaOutorga",how='left')

In [31]:
show_cases(df_usina,list_casos,return_all_columns=True)

,IdeUsinaOutorga,NomUsina,CodCegFormatado,DatMonitoramento,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra,IdcObraParalisada,IdcUsinaMonitorada,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,SigTipoGeracao,DscComercializacaoEnergia,DatInicioObraRealizado,DatMontagemRealizado,DscJustificativaPrevisao,CodLeilao,DatInicioSuprimento,CodCeg,DthEnvio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV,Usina_Sem_Rapeel,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT,classe,CC_real,ME_real,ValidadeAmbiental,CondicaoAmbiental,CondicaoConexao,CondicaoUso,PPA,criterio_novo,Homologar_Marcos,Dsc_Marcos_a_Homologar,Revisar_IO,Usina_Selecionada,Paralisada,Sem_Previsao,flagOPTeste30dias,UG_Sem_Rapeel,Prev_OC_passado,Em_teste,Sem_Monitoramento,Manual,Caso_I,Caso_II_a,Caso_II_b,Caso_III,Selecionado_BIU,criterionovopmo,dsccriterionovo,dscjustificativaregranova,Caso_I (case),Caso_II_a (case),Caso_II_b (case),Caso_III (case),Selecionado_BIU (case),Count
6,1928,Pacífico Mascarenhas,CGH.PH.MG.001928-3,2023-01-23 16:36:45.080,2021-09-20,1928-03-31,2021-09-20,2022-12-07,NaT,Não,True,2022-05-04,NaT,2023-07-29,2022-11-07,Em andamento,Não,2022-11-30,CGH,ACR,2021-09-21,2022-08-01,Estágio das obras da usina,04/2019,2025-01-01,CGHPHMG001928-3,2023-01-05 19:34:10.547,2021-09-20,NaT,2021-09-21,2021-09-20,2021-09-20,2022-11-07,2022-12-07,2022-11-30,NaT,NaT,NaT,2022-09-30,NaT,2022-05-04,False,NaT,2025-01-01,2025-01-01,2025-01-01,Válido,Não se Aplica,Válido,Não se Aplica,PH,2022-05-04,2022-09-30,2025-01-01,LO,OK,OK,ACR,0,False,,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,NaN,NaN,NaN,True,False,False,False,True,2098
0,324,Bugres,UHE.PH.RS.000324-7,2022-10-13 11:01:56.560,NaT,NaT,NaT,NaT,NaT,Não,True,NaT,NaT,2014-02-28,NaT,Não Iniciada,Sim,NaT,UHE,Fora do ACR,NaT,NaT,Inviabilidade da implantação da usina,NaN,NaT,UHEPHRS000324-7,2023-01-02 08:34:07.347,NaT,2025-02-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,False,NaT,2026-03-09,2002-12-11,NaT,Não se Aplica,Não se Aplica,Válido,Válido,PH,NaT,NaT,2026-03-09,LI,OK,OK,Nenhum,9,False,,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,9.0,Usina Viabilidade Baixa análise da fiscalização,Analisar justificativa: Revogação da outorga e...,False,False,False,False,False,169
8,2586,Salto Cafesoca,PCH.PH.AP.002586-0,2023-01-16 13:36:59.163,2022-10-03,2022-09-15,NaT,NaT,NaT,Não,True,NaT,NaT,2021-11-23,NaT,Em andamento,Não,NaT,PCH,ACR,2022-09-15,NaT,Estágio das obras da usina,01/2014,2021-03-30,PCHPHAP002586-0,2023-01-04 16:43:30.937,2022-10-03,NaT,2022-07-01,2022-07-01,NaT,2022-12-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,False,NaT,2024-06-22,2020-02-26,NaT,Não se Aplica,Não se Aplica,Não se Aplica,Não se Aplica,PH,NaT,NaT,2024-06-22,LI,Verificar,Verificar,ACR,0,True,DatSisTransmissaoRealizado,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,0.1,Usina em obras = Previsão OC maior entre data ...,Estágio das obras da usina,False,False,True,False,True,99
11,27243,Santa Cruz,UTE.GN.RJ.027243-4,2022-12-21 08:59:37.217,2018-04-02,NaT,NaT,2022-12-01,NaT,Não,True,NaT,NaT,NaT,2019-11-01,Em andamento,Não,2022-07-27,UTE,Fora do ACR,2018-04-02,2019-02-01,Estágio das obras da usina,NaN,NaT,UTEGNRJ027243-4,2023-01-05 14:22:33.463,2018-04-02,NaT,2018-04-02,NaT,NaT,2022-06-01,NaT,2022-07-27,NaT,2022-05-31,NaT,2019-02-01,NaT,NaT,False,NaT,2023-05-28,2021-03-07,2007-11-29,Não se Aplica,Válido,Válido,Válido,GN,NaT,2019-02-01,2007-11-29,LO,OK,OK,Nenhum,0,False,,False,True,False,False,True,False,False,True,False,True,False,F

In [32]:
raise

RuntimeError: No active exception to reraise

In [ ]:
# Renomeia colunas para exportação
dic = {
    'NumOperacaoUg':'NumUgUsina',
    'regranovapmo': 'PrevisaoOC_regra',
    'dscjustificativaregranova' : 'Justificativadaprevisao_new',
    'criterionovopmo' : 'CriterioPrevisao',
    'dsccriterionovo':'DscCriterioPrevisao',
    'Previsao_OC':'CalculoPrevisorOC',
    'Dat_OC_obrigacao':'OC_Obrigacao',
    'DscJustificativaPrevisao':'DscJustificativaPrevisaoAtual',
    'DatPrevisaoIniciobra':'prev_IO_SFG',
    'prev_IO':'prev_IO_rapeel'
}   


df_ug = df_ug.rename(columns=dic)
df_usina = df_usina.rename(columns=dic)

In [ ]:
""" ### include_on_script ###
print('BIU gerado.')
return (df_usina,df_ug)
"""
### end_function ###

In [ ]:
#### start_function ### function_name = export_biu_files
### start_function_vars ###
### start_funcion_defined_vars ###
### end_funcion_defined_vars ###
biu_path = biu_path
casos = ['I','I_a','BIU']

""" ### include_on_script ###
df_usina = df_usina
df_ug = df_ug
"""
### end_function_vars ###

if 'I' in casos:
    # Caso I
    file_name = os.path.join(biu_path,f"Caso_I.xlsx")
    df_ug[df_ug.Caso_I][['IdeUsinaOutorga','DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']].to_excel(file_name)

# Caso II_a
if 'II_a' in casos:
    file_name = os.path.join(biu_path,f"Caso_II_a.xlsx")
    df_usina[df_usina.Caso_II_a][['IdeUsinaOutorga','DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']].to_excel(file_name)
if 'BIU' in casos:
    # Caso II-b e III
    file_name = os.path.join(biu_path,f"BIU.xlsx")
    df_usina[df_usina.Caso_II_b | df_usina.Caso_III].to_excel(file_name)
### end_function ###

## Análise de casos BIU

In [ ]:
list_criterios_BIU = ['Manual', 'Revisar_IO','Homologar_Marcos', 'Em_teste','Sem_Monitoramento']
casos = df_usina[list_criterios_BIU +  list_casos].value_counts().reset_index().sort_values(by='Caso_III')
casos
mask_casos_selecionados = casos[list_casos].any(axis=1)
casos[mask_casos_selecionados]

In [ ]:
df_usina.loc[df_usina.Selecionado_BIU,list_casos+ ['Manual','Homologar_Marcos']].value_counts().reset_index()

In [ ]:
#df_usina[df_usina.Homologar_Marcos & (~ df_usina.Sem_Monitoramento)][['IdeUsinaOutorga','NomUsina','Homologar_Marcos',"Sem_Monitoramento",'DatMonitoramento','DthEnvio','Dsc_Marcos_a_Homologar']].to_excel("Usinas_sem_fotos.xlsx",index=False)

In [ ]:
raise

# Testes

## Junta dados do SAS e Python

In [ ]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})[["IdeUsinaOutorga","NumOperacaoUg",'CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo','CalculoPrevisorOC_mx',"FASE"]]

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True


#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [ ]:
criterios_da_previsao_sas_gb = criterios_da_previsao_sas.groupby("IdeUsinaOutorga")[['CriterioNovoPMO','DscCriterioNovo','DscJustificativaRegranova']].nunique()
usinas_dois_criterios = criterios_da_previsao_sas_gb[criterios_da_previsao_sas_gb.CriterioNovoPMO>1].index

In [ ]:
criterios_da_previsao_sas_usina = criterios_da_previsao_sas[['IdeUsinaOutorga','CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo']].drop_duplicates()

In [ ]:
merge_usina  = pd.merge(df_usina_criterio[['IdeUsinaOutorga','criterionovopmo','dscjustificativaregranova','dsccriterionovo']],
criterios_da_previsao_sas_usina,on=['IdeUsinaOutorga'],suffixes=('_py','_sas'),how='outer')

In [ ]:
criterios_da_previsao_sas_ug = criterios_da_previsao_sas[list_id_ug + ['CalculoPrevisorOC_mx','FASE']]

In [ ]:
merge_ug = pd.merge(df_ug[list_id_ug + ['FASE','Previsao_OC']],criterios_da_previsao_sas_ug,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge_ug['OC_diff'] = merge_ug.Previsao_OC - merge_ug.CalculoPrevisorOC_mx

## Teste previsão

In [ ]:
merge_ug.sort_values(by='OC_diff',ascending=False).head(20)

## Teste critérios

### FASE ok

In [ ]:
merge_ug[merge_ug.FASE_py != merge_ug.FASE_sas]

### criterionovopmo OK

In [ ]:
merge_usina[(merge_usina.CriterioNovoPMO != merge_usina.criterionovopmo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

In [ ]:
merge_usina[(merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))].sample(6)

### dscjustificativaregranova OK

In [ ]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]


In [ ]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova)]

### dsccriterionovo OK

In [ ]:
merge_usina[(merge_usina.dsccriterionovo != merge_usina.DscCriterioNovo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]